In [23]:
import skinematics as skin
import numpy as np
import treecorr

#import random

In [2]:
def spherical_to_cartesian(P_sph, sph_units):
    # r , theta, phi 
    r = P_sph[0]
    theta = 0
    phi = 0
    if (sph_units == 'deg'):
        theta = np.deg2rad(P_sph[1])
        phi = np.deg2rad(P_sph[2])
    elif (sph_units == 'rad'):
        theta = P_sph[1]
        phi = P_sph[2]  
    else:
        raise Exception('units should be either \'deg\' or \'rad\'')
        
    return np.array([r*np.sin(theta)*np.cos(phi),r*np.sin(theta)*np.sin(phi),r*np.cos(theta)])

In [3]:
def cartesian_to_spherical(P_cart, sph_units):
    # x, y, z ---> r, theta, phi 
    x = P_cart[0]
    y = P_cart[1]
    z = P_cart[2]
    
    r = np.sqrt(x*x+y*y+z*z)
    
    theta = np.arctan2(np.sqrt(x*x+y*y), z)
        
    phi = np.arctan2(y,x)
    if phi < 0:
        phi = 2*np.pi + phi
    
    if (sph_units == 'deg'):
        return np.array([r, theta*180/np.pi, phi*180/np.pi])
    elif (sph_units == 'rad'):
        return np.array([r, theta, phi])
    else:
        raise Exception('units should be either \'deg\' or \'rad\'')

In [4]:
def rotate_vector_to_NP(P_sph, P_sph_units):
    theta = 0
    phi = 0
    if (P_sph_units == 'deg'):
        theta = P_sph[1]
        phi = P_sph[2]         
    elif (P_sph_units == 'rad'):
        theta = np.rad2deg(P_sph[1])
        phi = np.rad2deg(P_sph[2])
    else:
        raise Exception('units should be either \'deg\' or \'rad\'')
        
    P_cart = spherical_to_cartesian(P_sph, P_sph_units)

    # First rotate about z axis (counter azimuthally) so as to make the point lie in the xz plane i.e. y = 0
    Rot_z = skin.rotmat.R(axis='z', angle=-phi) # cskin.rotmar.R only accepts angle in degrees!   
    P_cart_xz = np.matmul(Rot_z, P_cart)
    
    # Then rotate about y axis (counter polar-wise) to make the point lie at the NP i.e. x = 0, y = 0
    Rot_y = skin.rotmat.R(axis='y', angle=-theta) # cskin.rotmar.R only accepts angle in degrees!
    P_cart_NP = np.matmul(Rot_y, P_cart_xz) 
       
    return P_cart_NP

In [31]:
def rotate_vector_to_NP_matrices(P_sph, P_sph_units):
    theta = 0
    phi = 0
    if (P_sph_units == 'deg'):
        theta = P_sph[1]
        phi = P_sph[2]         
    elif (P_sph_units == 'rad'):
        theta = np.rad2deg(P_sph[1])
        phi = np.rad2deg(P_sph[2])
    else:
        raise Exception('units should be either \'deg\' or \'rad\'')
        
    # First rotate about z axis (counter azimuthally) so as to make the point lie in the xz plane i.e. y = 0
    Rot_z = skin.rotmat.R(axis='z', angle=-phi) # cskin.rotmar.R only accepts angle in degrees!   
    
    # Then rotate about y axis (counter polar-wise) to make the point lie at the NP i.e. x = 0, y = 0
    Rot_y = skin.rotmat.R(axis='y', angle=-theta) # cskin.rotmar.R only accepts angle in degrees!
       
    return np.array([Rot_z, Rot_y]) # 1st rotation matrix and then 2nd rotation matrix

In [32]:
P = np.array([2, 90, 45])

In [33]:
spherical_to_cartesian(P, 'deg')

array([1.41421356e+00, 1.41421356e+00, 1.22464680e-16])

In [34]:
cartesian_to_spherical(rotate_vector_to_NP(P, 'deg'), 'deg')

array([2., 0., 0.])

In [35]:
R_1, R_2 = rotate_vector_to_NP_matrices(P, 'deg')

In [36]:
R_1

array([[ 0.70710678,  0.70710678,  0.        ],
       [-0.70710678,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [37]:
R_2

array([[ 6.123234e-17,  0.000000e+00, -1.000000e+00],
       [ 0.000000e+00,  1.000000e+00,  0.000000e+00],
       [ 1.000000e+00,  0.000000e+00,  6.123234e-17]])

In [40]:
cartesian_to_spherical(np.matmul(R_1.T, np.matmul(R_2.T, rotate_vector_to_NP(P, 'deg'))), 'deg')

array([ 2., 90., 45.])

In [12]:
def draw_pt_on_circle(omega_1, theta_scale, patch_radius):
    R_1, R_2 = rotate_vector_to_NP_matrices(omega_1, 'rad')
    
    theta_2_prime = theta_scale
    
    do_iteration = True
    
    while(do_iteration):
        phi_2_prime = np.random.uniform(0, 2*np.pi, 1)
        omega_2_prime_cart = spherical_to_cartesian([theta_2_prime, phi_2_prime], 'rad')
        omega_2 = np.matmul(R_1.T, np.matmul(R_2.T, omega_2_prime_cart))
    
        if (omega_2[0] < patch_radius):
            do_iteration = False
            
    return omega_2

In [15]:
np.random.uniform(0, 2*np.pi, 1)

array([4.71835185])

In [25]:
kk = treecorr.KKCorrelation(min_sep=1, max_sep=400, nbins=30, sep_units='arcmin')
theta_tc = kk.meanr

In [26]:
theta_tc

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [27]:
kk = treecorr.KKCorrelation(min_sep=1, max_sep=400, nbins=30, sep_units='arcmin')
theta_scale_log_vec = kk.logr # in log arcmins

In [43]:
np.exp(theta_scale_log_vec)*np.pi/180/60

array([0.00032144, 0.00039249, 0.00047925, 0.00058519, 0.00071455,
       0.00087251, 0.00106538, 0.00130089, 0.00158846, 0.0019396 ,
       0.00236836, 0.00289189, 0.00353116, 0.00431175, 0.00526488,
       0.00642871, 0.00784981, 0.00958505, 0.01170388, 0.01429108,
       0.0174502 , 0.02130766, 0.02601783, 0.03176921, 0.03879196,
       0.04736713, 0.05783789, 0.07062326, 0.08623491, 0.10529759])

In [29]:
kk = treecorr.KKCorrelation(min_sep=1, max_sep=400, nbins=30, sep_units='arcmin')
theta_tc = kk.meanr